In [ ]:
# Install required packages
! pip install folium plotly pydeck

# Import Required Libraries
import pandas as pd
import folium
from folium.plugins import HeatMap
import plotly.express as px


# Manually upload and read file
df = pd.read_csv('cleaned_data.csv')

# Data cleaning describing type data
print(f"Original rows: {len(df)}")

# creating a slice of columns for easier plotting
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

print(f"Rows after cleaning: {len(df)}")

print(f"\nTrash amount statistics:")
print(df['Trash amount'].describe())


Original rows: 189
Rows after cleaning: 189

Trash amount statistics:
count      189.000000
mean       541.158730
std       1465.665859
min         14.000000
25%        116.000000
50%        227.000000
75%        533.000000
max      18174.000000
Name: Trash amount, dtype: float64


FOLIUM Heatmap (interarctive - output is downloaded directly to user's computer)

In [ ]:
# Creating an interactive heatmap with Folium
# Documentation: https://python-visualization.github.io/folium/latest/getting_started.html

# Added colab.files as an optionto download file directly to pc
from google.colab import files

m = folium.Map(
    location=[41.8719, 12.5674],
    zoom_start=6,
    tiles='CartoDB positron' # Can write diffferent styles ex. Open Street Maps
)

heat_data = [[row['lat'], row['lon'], row['Trash amount']]
             for idx, row in df.iterrows()]

HeatMap(
    heat_data,
    min_opacity=0.2,
    max_opacity=0.9,
    radius=20,
    blur=20,
    gradient={
        0.2: 'blue',
        0.4: 'cyan',
        0.6: 'lime',
        0.8: 'yellow',
        1.0: 'red'
    }
).add_to(m)

# Add markers for the top polluted beaches
top_beaches = df.nlargest(10, 'Trash amount')

for idx, row in top_beaches.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=8,
        popup=f"{row['Original_Beach_Name']}<br>Trash: {row['Trash amount']:.0f}", # Use popup for marking to be easily distinguishable
        color='darkred',
        fill=True,
        fillColor='red'
    ).add_to(m)

display(m)

m.save('it_folium_heatmap.html') # Save as an .html extension for easy handling and collaboration
files.download('it_folium_heatmap.html') # called from the google.colab dependency


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PLOTLY density map with a legend of trash amount

In [ ]:
# Creating a density map by using Plotly

fig = px.density_mapbox(
    df,
    lat='lat',
    lon='lon',
    z='Trash amount',
    radius=25,
    center=dict(lat=41.8719, lon=12.5674),
    zoom=5,
    mapbox_style='carto-positron', # again can give different values, open street maps
    color_continuous_scale='Portland',
    title='Trash Density Heatmap - Italian Beaches',
    labels={'Trash amount':  'Trash Items'},
    hover_data={
        'Original_Beach_Name': True, # name is displayed while hovering around
        'Trash amount': ':.0f',
        'lat': ':.3f',
        'lon': ':.3f'
    }
)

fig.update_layout(
    height=700,
    coloraxis_colorbar=dict(
        title="Trash<br>Items",
        thicknessmode="pixels",
        thickness=15,
        lenmode="pixels",
        len=300
    )
)

fig.show()

# Download plot as html extension and save to computer
fig.write_html('it_plotly_density.html')
files.download('it_plotly_density.html')







<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PLOTLY animated seasonality density map with slider and legend of trash amount

In [ ]:
# Creating a seasonality density based on highest amount of trash
# Created with the help of GitHub Copilot
# Very fun interacting with, nice plotting experience very different in what we are used to.


# Animated heatmap that changes by time period (cleaned_data)
# Allows for play/pause function and manual sliding of bar

fig_seasonal = px.density_mapbox(
    df,  # Full dataset with all time periods

    # Geographic columns
    lat='lat',
    lon='lon',
    z='Trash amount',

    # Creates slider/play button for different periods
    animation_frame='Period',  # Column containing time periods (seasons)
    # Each unique value in 'Period' becomes one animation frame

    # Visual parameters
    radius=25,

    # Map settings
    center=dict(lat=41.8719, lon=12.5674),
    zoom=5,
    mapbox_style='carto-positron',

    # Color scheme
    color_continuous_scale='Purples',  # Purple to fit story map scheme

    # Title
    title='Seasonal Trash Density'
)

# Set plot height
fig_seasonal.update_layout(height=700)

# Display with animation controls
fig_seasonal.show()

# Save to html file and download in computer
fig_seasonal.write_html('it_plotly_seasonal.html')
files.download('it_plotly_seasonal.html')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---WHAT WENT WELL---

1. This script accurately creates and visualizes 3 different kind of plots allowing us to get more familiar with more visualization techniques based on plotly, plotly.express and folium

2. Clear documentation found for plotly animated map and basic heatmap, allows to understand and explore different options

3. Creating an .html file of folium, allows easy sharing and collaboration in google drive and team's github; providing also some familiarity to less experienced team member.

4. Features like highlighting top polluted beaches in the animated plot allowed us to explore temporal patterns of trash distribution.

--------------------------------------------------------------------------------

---WHAT WENT BADLY---

 ISSUES AND PROBLEMS:

1. Repetitive Code: The same data loading and cleaning code appears in multiple cells. Should be defined once at the top.

2. Hard-coded Values: Map center coordinates [41.8719, 12.5674] are hard-coded everywhere. Should be calculated from data or defined as constants.

3. Data quality and portability concerns arise with continuous incosistent numeric formats and colab-specific downloadable codew.

-------------------------------------------------------------------------------

---WHAT COULD BE IMPROVED---

1. Add a try-except code block to prevent code from crashing if data is corrupted or code is miss written. Providing proper {error handling} could give usefull error messages as an output and gemini help would identify missing values in and out of code.

2. In the animated seasonal trend we could add fine scale detailing over the amount of litter found in the beaches. For now there is no analysis of how trash amount change in a particular beach.

3. Add a dict plus resusable fuctions where could read the same information (map, zoom, extent, centre) to reduce the **DRY** (Don't Repeat Yourself) principle of programming.

4. If wanting to create a stroy map code integration within arc online would suffice rather sharing and publishing colab.file in the case of maps, plots and graphs.


  